In [ ]:
import pandas as pd
import numpy as np
import os
import re
from pandas import ExcelWriter
from datetime import datetime as dt

In [ ]:
#Грузим отчет по диспутам
for filename in os.listdir('.'):
    if (filename.startswith('Report') or filename.startswith('Export')) and (filename.endswith('.xlsm') or filename.endswith('.xlsx')):
        df = pd.read_excel('./' + filename, sheet_name='Sheet1')
df['Номер терминала'] = df['Номер терминала'].str.strip('\r\n ') #убираем пробелы в номере терминала

In [ ]:
#Убираем бины из названий банков, заменяем в Сумме диспута запятые на точки, задаем Дате старта БП формат даты
df['Эквайрер'] = df['Эквайрер'].str.replace('\d{6,} - ', '')
df['Эмитент'] = df['Эмитент'].str.replace('\d{6,} - ', '')

In [ ]:
for i in range(df.shape[0]):
        row = df['Эквайрер'].iloc[i]
        if re.search('/d{4}', row):
            df['Эквайрер'].iloc[i] = re.sub('/\d{4}', '', row)

In [ ]:
#Задаем Сумме диспута формат числа, записываем данные в файл на лист raw_data для формирования сводной таблицы (при необходимости)
df['Сумма диспута'] = pd.to_numeric(df['Сумма диспута'])
with ExcelWriter('./DISPUTES_MIR.xlsx', engine="openpyxl", mode='w') as writer: 
    df.to_excel(writer, sheet_name="raw_data", index = False)

In [ ]:
#Формируем датафрейм по заданным условиям, считаем сумму и количество диспутов, сводим вместе, сортируем по убыванию, добавляем столбец с комментарием
month = int(input('Указать номер нужного месяца: '))
df_disp = df[(df['Дата старта БП'].dt.month == month)& #За текущий месяц (или указать номер нужного месяца)
             (df['ПС'] == 'МИР')& #ПС "Мир"
             (df['Вид деятельности'] == 'Эквайринг')& #Эквайринг
             (df['Направление БП'] == 'Входящий БП')& #Входящие БП
             ((df['Тип БП'] =='Жалоба')|(df['Тип БП'] == 'Диспут'))] #Типы БП: Жалоба и Диспут
df_sum = df_disp.groupby('Эквайрер')['Сумма диспута'].sum()
df_count = df_disp.groupby('Эквайрер')['Индекс БП'].count()
df_itog = pd.concat([df_sum, df_count], axis = 1)
df_itog.reset_index(inplace= True )
df_itog.columns = ['Эквайер', 'Сумма диспутов, р', 'Количество диспутов']
df_itog = df_itog.sort_values(by='Сумма диспутов, р', ascending=False).reset_index(drop = True)
df_itog.loc[df_itog['Сумма диспутов, р'] > 500000, 'Нарушение'] = 'Сумма превышена'
df_itog.loc[df_itog['Количество диспутов'] > 40, 'Нарушение'] = 'Количество превышено'
df_itog.loc[(df_itog['Сумма диспутов, р'] > 500000)&(df_itog['Количество диспутов'] > 40), 'Нарушение'] = 'Сумма и количество превышены'
df_itog.loc[(df_itog['Сумма диспутов, р'] <= 500000)&(df_itog['Количество диспутов'] <= 40), 'Нарушение'] = 'Превышений нет'

In [ ]:
#Выделяем высокорисковые ТСП
df_disp_HR = df_disp.loc[df_disp['МСС'].isin([4511, 4722, 5122, 5912, 5921, 5993, 6012, 6051, 6211, 7273, 7995, 6537])|
            ((df_disp['МСС'] >= 3000)&(df_disp['МСС'] <= 3300))]

In [ ]:
#Записываем точки эквайеров с нарушением, по которым пришли диспуты, на лист TSP
acq_list = df_itog.loc[df_itog['Нарушение'] != 'Превышений нет', 'Эквайер']
for i in acq_list:
    df_sum2 = df_disp[df_disp['Эквайрер'] == i].groupby('Номер терминала')['Сумма диспута'].sum()
    df_count2 = df_disp[df_disp['Эквайрер'] == i].groupby('Номер терминала')['Индекс БП'].count()
    df_itog2 = pd.concat([df_sum2, df_count2], axis = 1)
    df_sum2_HR = df_disp_HR[df_disp_HR['Эквайрер'] == i].groupby('Номер терминала')['Сумма диспута'].sum()
    df_count2_HR = df_disp_HR[df_disp_HR['Эквайрер'] == i].groupby('Номер терминала')['Индекс БП'].count()
    df_itog2 = pd.concat([df_itog2, df_sum2_HR], axis = 1)
    df_itog2 = pd.concat([df_itog2, df_count2_HR], axis = 1)    
    df_itog2.reset_index(inplace= True )
    df_itog2.columns = ['Номер терминала', 'Сумма диспутов, р', 'Количество диспутов', 'Сумма диспутов в рисковых ТСП, р', 'Количество диспутов в рисковых ТСП']
    df_itog2 = df_itog2.sort_values(by='Сумма диспутов, р', ascending=False).reset_index(drop = True)
    df_itog2['Эквайер'] = i
    df_itog2 = df_itog2.merge(df_disp[['Номер терминала', 'Точка обслуживания', 'МСС']].drop_duplicates(), how = 'left', on = 'Номер терминала')
    with ExcelWriter('./DISPUTES_MIR.xlsx', engine="openpyxl", mode='a') as writer: 
        df_itog2.to_excel(writer, sheet_name= "TSP")

In [ ]:
#Записываем полученные показатели на лист month_stat, где month - текущий (указанный) месяц
month_list = ['january', 'february', 'march', 'april', 'may', 'june',
           'july', 'august', 'september', 'october', 'november', 'december']
with ExcelWriter('./DISPUTES_MIR.xlsx', engine="openpyxl", mode='a') as writer: 
    df_itog.to_excel(writer, sheet_name= month_list[month - 1] + " _stat")

In [ ]:
#Диспуты РНКО
df_RNKO = df_disp[df_disp['Эквайрер'] == 'РНКО "Платежный Центр" (ООО)']
df_RNKO.to_excel('./DISPUTES_RNKO.xlsx')

In [ ]:
for i in range(df.shape[0]):
        row = df['Эквайрер'].iloc[i]
        if re.search('/d{4}', row):
            df['Эквайрер'].iloc[i] = re.sub('/\d{4}', '', row)